In [1]:
import boto3
import spotipy
import json
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials
import numpy as np

In [ ]:
# TODO
# Artist seed needs to search song first and get value that best matches target value 
# Normalize values
# TODO handle case where recommendation doesn't return anything

In [2]:
client = boto3.client('lambda')
response = client.invoke(FunctionName='spotifyLambda', InvocationType='RequestResponse')
data = json.loads(response['Payload'].read())
token = json.loads(data['body'])['token']

In [3]:
sp = spotipy.Spotify(token)

In [4]:
def getFeatures(songID):
    audioFeaturesRaw = sp.audio_features(songID)[0]    
    return audioFeaturesRaw

In [5]:
def getRecommendations(ids, **kwargs):
#     print("Recommendations: ", ids, kwargs)
    recommendations = sp.recommendations(seed_tracks=ids, **kwargs)
    
    return recommendations['tracks']

In [16]:
def generatePlaylist(numTracks, targets, seed):
    import copy
    actual = copy.deepcopy(targets)
    
    """
    numTracks: number of tracks in the set
    targets: dictionary of target values
    seeds: dictionary of spotify id of artist or song, and location
    """
    playlist = np.empty(shape=numTracks, dtype=np.object)
    
    seedIndex = seed['location']
    features = getFeatures(seed['id'])
    playlist[seedIndex] = seed['id']
    for metric in targets:
        if metric == 'target_instrumentalness':
            continue
        targets[metric] = np.asarray(targets[metric], dtype=np.double)
        features = getFeatures(seed['id']) 
        offset = features[metric.split('_')[1]] - targets[metric][seedIndex]
        if offset > 0:
            targets[metric] = targets[metric]+offset
        if offset < 0:
            targets[metric] = targets[metric]-offset
    
            
    for i in range(seedIndex+1, len(playlist)):
        
        k = {}
        for metric in targets:
            k[metric] = targets[metric][i]        
        num_seeds = min(i-seedIndex, 5)
        trackseeds = playlist[seedIndex:seedIndex+num_seeds]
        recommendations = getRecommendations(list(trackseeds), **k)
        if len(recommendations)==0:
            print("No Recs")
        for track in recommendations:
            feature_val_diff = -1
            if track['id'] not in playlist:
                playlist[i] = track['id']
                break

    for i in range(seedIndex-1, -1, -1):
        
        k = {}
        for metric in targets:
            k[metric] = targets[metric][i]        
        x = len(playlist)-i
        if x >= 5:
            x = 5
        trackseeds = playlist[i+1:i+x+1]
        recommendations = getRecommendations(list(trackseeds), **k)
        if len(recommendations)==0:
            print("No Recs")
        for track in recommendations:
            if track['id'] not in playlist:
                playlist[i] = track['id']
                break
    
#     actual = []
#     for i in range(0, len(playlist)):
#         features = getFeatures(playlist[i])
#         actual.append(features['danceability'])
#     print(targets['target_danceability'])
#     print(actual)
    
#     actual = []
#     for i in range(0, len(playlist)):
#         features = getFeatures(playlist[i])
#         actual.append(features['energy'])
#     print(targets['target_energy'])
#     print(actual)
    resultAnalysis(playlist, targets)
    return playlist  

In [10]:
def resultAnalysis(playlist, targets):
    
    features = []
    
    for i in range(0, len(playlist)):
        features.append(getFeatures(playlist[i]))
        
    for metric in targets:
        actual = []
        for j in range(0, len(targets[metric])):
            actual.append(features[j][metric.split('_')[1]])
        print(f"TARGET {metric.split('_')[1]}: {targets[metric]}")
        print(f"ACTUAL {metric.split('_')[1]}: {actual}")
        
        

In [17]:
danceability = [.60, .65, .70, .92, .73, .65]
energy = [.50, .60, .65, .82, .65, .50]
valence = [.75, .60, .65, .75, .65, .65]
instrumentalness = [1.0, 0.0, 0.0, 1.0, 0.0, 1.0]
targets= {
        'target_danceability': danceability,
        'target_energy': energy,
        'target_valence': valence,
        'target_instrumentalness': instrumentalness
}
seeds = {
    'location': 2,
    'id': '2WJMb5llcepSZ2mKRD6Frt'

}
playlist = generatePlaylist(len(danceability), targets, seeds)

TARGET danceability: [0.621 0.671 0.721 0.941 0.751 0.671]
ACTUAL danceability: [0.775, 0.718, 0.679, 0.877, 0.77, 0.545]
TARGET energy: [0.729 0.829 0.879 1.049 0.879 0.729]
ACTUAL energy: [0.798, 0.84, 0.879, 0.854, 0.808, 0.837]
TARGET valence: [1.3214 1.1714 1.2214 1.3214 1.2214 1.2214]
ACTUAL valence: [0.869, 0.95, 0.0786, 0.537, 0.885, 0.734]
TARGET instrumentalness: [1.0, 0.0, 0.0, 1.0, 0.0, 1.0]
ACTUAL instrumentalness: [0.903, 0.00076, 0.773, 0.879, 0.00163, 0.926]


In [38]:
sp.user_playlist_create(name="Test Playlist", user="jup118")

{'collaborative': False,
 'description': '',
 'external_urls': {'spotify': 'https://open.spotify.com/playlist/0qxanOC7s6PZeai31xM2hi'},
 'followers': {'href': None, 'total': 0},
 'href': 'https://api.spotify.com/v1/playlists/0qxanOC7s6PZeai31xM2hi',
 'id': '0qxanOC7s6PZeai31xM2hi',
 'images': [],
 'name': 'Test Playlist',
 'owner': {'display_name': 'jup118',
  'external_urls': {'spotify': 'https://open.spotify.com/user/jup118'},
  'href': 'https://api.spotify.com/v1/users/jup118',
  'id': 'jup118',
  'type': 'user',
  'uri': 'spotify:user:jup118'},
 'primary_color': None,
 'public': True,
 'snapshot_id': 'MSw0YjM4NzA1ZGNhNThjMjk2Njg2MGQxNGE2MGU3ZDUwYmY0N2U1OGJh',
 'tracks': {'href': 'https://api.spotify.com/v1/playlists/0qxanOC7s6PZeai31xM2hi/tracks',
  'items': [],
  'limit': 100,
  'next': None,
  'offset': 0,
  'previous': None,
  'total': 0},
 'type': 'playlist',
 'uri': 'spotify:playlist:0qxanOC7s6PZeai31xM2hi'}

In [18]:
sp.playlist_add_items(items=playlist, playlist_id="0qxanOC7s6PZeai31xM2hi")

{'snapshot_id': 'MTEsNzU2ZjhhMjRiOGQ4NDM3N2ZjY2Y3MjYzMzAxNTIxMzcwMzc1MTY1ZA=='}

In [56]:
playlist

array(['3ioD2gvfwj4YoOKhxS1EgN', 'spotify:track:22bNlvnDQbYAq0Ex2sduSu',
       '1LxenWKrYzbX8zvbuZgKHn', '6IgIPWjUrRBTIm07t7H64S',
       '5esTnUp3cLgKsU2Etci2Ao'], dtype=object)